# Getting Data from World Bank

In [18]:
pip install wbdata

Note: you may need to restart the kernel to use updated packages.


In [19]:
import wbdata
import pandas as pd
import sys
import io
import pickle
from threading import Thread
from time import time

In [20]:
stdout = sys.stdout
sys.stdout = io.StringIO()
wbdata.get_indicator()
indicators = sys.stdout.getvalue()
sys.stdout = stdout

In [21]:
ind_temp = indicators.split('\n')
ind_temp_2 = [i.split('\t') for i in ind_temp]
#print(ind_temp_2)

In [22]:
ind_temp = indicators.split('\n')
ind_temp_2 = [i.split('\t') for i in ind_temp]
for i in range(len(ind_temp_2)):
    ind_temp_2[i][0]=ind_temp_2[i][0].strip()
#print(ind_temp_2)

In [23]:
d = {}
for i in ind_temp_2:
    try:
        d[i[0]]=i[1]
    except:
        pass

In [18]:
d

{'1.0.HCount.1.90usd': 'Poverty Headcount ($1.90 a day)',
 '1.0.HCount.2.5usd': 'Poverty Headcount ($2.50 a day)',
 '1.0.HCount.Mid10to50': 'Middle Class ($10-50 a day) Headcount',
 '1.0.HCount.Ofcl': 'Official Moderate Poverty Rate-National',
 '1.0.HCount.Poor4uds': 'Poverty Headcount ($4 a day)',
 '1.0.HCount.Vul4to10': 'Vulnerable ($4-10 a day) Headcount',
 '1.0.PGap.1.90usd': 'Poverty Gap ($1.90 a day)',
 '1.0.PGap.2.5usd': 'Poverty Gap ($2.50 a day)',
 '1.0.PGap.Poor4uds': 'Poverty Gap ($4 a day)',
 '1.0.PSev.1.90usd': 'Poverty Severity ($1.90 a day)',
 '1.0.PSev.2.5usd': 'Poverty Severity ($2.50 a day)',
 '1.0.PSev.Poor4uds': 'Poverty Severity ($4 a day)',
 '1.1.HCount.1.90usd': 'Poverty Headcount ($1.90 a day)-Rural',
 '1.1.HCount.2.5usd': 'Poverty Headcount ($2.50 a day)-Rural',
 '1.1.HCount.Mid10to50': 'Middle Class ($10-50 a day) Headcount-Rural',
 '1.1.HCount.Ofcl': 'Official Moderate Poverty Rate- Rural',
 '1.1.HCount.Poor4uds': 'Poverty Headcount ($4 a day)-Rural',
 '1.1.H

In [8]:
temp = wbdata.get_data(1)

TypeError: sequence item 3: expected str instance, int found

In [9]:
def get_data(i,table_list,id_value_dict,error_list):
    '''
    i: identifier
    table_list: a list to store data from WorldBank
    id_value_dict: a dictionary to store id,value pair. For some identifiers with the same id, the id,value pair retrieved from get_data()
                   is different from the id,value pair retrieved from get_indicator(). Store the id,value pair from get_data() in case needed.
    error: a list to store identifiers failed to fetch data
    '''
    global n
    try:
        temp = wbdata.get_data(i)
        id_value_dict[temp[0]['indicator']['id']] = temp[0]['indicator']['value']
        for i in range(len(temp)):
            temp[i]['indicator']=temp[i]['indicator']['value']
            temp[i]['country']=temp[i]['country']['value']            
        table = pd.DataFrame.from_dict(temp).dropna()
        table = table.set_index(['country','date']).sort_index()
        table = table.rename(columns={'value':table.indicator[0]})
        table = pd.DataFrame(table[table.indicator[0]])
        table_list.append(table)
        n+=1
        print(n,'done',end='...',flush=True)
    except:
        error_list.append(i)
        n+=1
        print(n,'done',end='...',flush=True)

In [10]:
len(d)

17016

In [11]:
ind = list(range(0,17201,100))+[len(d)]
for j in range(len(ind)-1):
    id_value_dict = {}
    error_list = []
    table_list = []
    threadlist = []
    print('get data for ',ind[j],'-',ind[j+1],'...')
    n = 0
    for i in list(d.keys())[ind[j]:ind[j+1]]:
        t = Thread(target=get_data,args=(i,table_list,id_value_dict,error_list))
        t.start()
        threadlist.append(t)
    for i in threadlist:
        i.join()
    with open(''.join(['data/data_',str(ind[j]),'.pkl']),'wb') as f:
        pickle.dump(table_list,f)
    with open(''.join(['data/dict_',str(ind[j]),'.pkl']),'wb') as f:
        pickle.dump(id_value_dict,f)
    with open(''.join(['data/error_',str(ind[j]),'.pkl']),'wb') as f:
        pickle.dump(error_list,f)
    print('\n',ind[j],'-',ind[j+1],'done')

get data for  0 - 100 ...
12  done...done...3 done...45 done... done...6 done...7 done...8 done...91011 done... done... done...12 done...13 done...14 done...15 done...161718 done... done... done...19 done...20 done...21 done...22 done...23 done...24 done...25 done...26 done...27 done...28 done...2930 done... done...31 done...32 done...33 done...34 done...35 done...36 done...37 done...38 done...39 done...40 done41 done......42 done...43 done...44 done...45 done...46 done...47 done...48 done...49 done...5051 done... done...52 done...53 done...54 done...55 done...56 done...5758 done59 done...... 60done... done...61 done...6263 done... done...64 done...65 done...6667 68done... done...69 done... done...70 done...71 done...72 done...73 done...74 done75 done......76 done...77 78done79 done... done......80 done...8182 done... done83 done......8485 done... 86 done...done87 done......88 done...89 done...90 done...91 done...92 done...93 done...94 done...95 done...96 done...97 done...98 done...99 

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_0.pkl'

# Merge tables

In [12]:
import pickle
import os
import pandas as pd

In [14]:
try:
    os.mkdir('final')
except:
    pass
folder = 'final/'
files = os.listdir(folder)

In [15]:
# merge tables for every pkl file
table_list = []
print('\ntotal files: ',len(files))
for i,pkl_name in enumerate(files):
    with open(''.join([folder,pkl_name]),'rb') as f:
        l_temp = pickle.load(f)
    table = l_temp[0]
    if not l_temp:
        for table_temp in l_temp[1:]:
            table = table.merge(table_temp,how='outer',on=['country','date'])
    table_list.append(table)
    print(i,'done',end=' ... ')


total files:  0


In [16]:
# merge all tables in the list
print('\ntotal tables: ',len(table_list))
table = table_list[0]
for i,table_temp in enumerate(table_list)[1:]:
    table = table.merge(table_temp,how='outer',on=['country','date'])
    print(i,'done',end=' ... ')


total tables:  0


IndexError: list index out of range